# Battleship

01-October-2023 KPC



In [ ]:
# This is a class definition so it needs to be in one cell.

class Battleship:
    """Manages a Battleship game board of size specified at construction.
    The board is represented as a grid of cells where ships can be placed,
    and players can guess the coordinates to hit the opponent's ships.
    The contents of a cell are referred to as the 'mark', where ' ' represents empty sea,
    'X' represents a ship, and 'H' represents a hit on a ship."""

    # CONSTRUCTOR
    def __init__(self,n=10):
        """Constructor - creates and initializes a Battleship board of size n x n.
        All cells are initialized to ' ' (empty sea). The default size is 10x10."""
        self.n = n
        self.my_board = [ [' ' for _ in range(self.n)] for __ in range(self.n)]
        self.ai_board = [ [' ' for _ in range(self.n)] for __ in range(self.n)]
        self.guess_board = [ [' ' for _ in range(self.n)] for __ in range(self.n)]
        return

    # REPRESENTATIONS
    def fws(self,thing,width):
        """fixed-width-string utility func.
        It returns str(thing), exactly width characters wide, padded on left by spaces.
        If str(thing) is itself wider than 'width', it gets truncated to the rightmost width chars.
        """
        return (' '*width + str(thing))[-width:]

    def __str__(self,board):
        """Pretty-printable string representation of a Battleship board."""
        if (board == 'User'):
            board = self.my_board
            label = 'User Board'
        elif (board == 'AI'):
            board = self.ai_board
            label = 'AI Board'
        else:
            board = self.guess_board
            label = 'Guess Board'

        # For each row of the board, two lines in the output.
        boardlines = ''

        s = ''
        maxwidth = len(str(self.n))
        numstrs = [self.fws(i,maxwidth) for i in range(self.n)]
        # First line is column numbers
        firstline = ' '*(maxwidth) + ''.join([' '+ self.fws(i,maxwidth) for i in range(self.n)])

        for i in range(self.n):
            boardlines += self.fws(i, maxwidth) + ' ' + self.fws(board[i][0], maxwidth)
            for j in range(1, self.n):
                boardlines += '|' + self.fws(board[i][j], maxwidth)
            boardlines += '\n'
            if i < (self.n - 1):
                boardlines += ' ' + ' ' * maxwidth + '-' * maxwidth + ('|' + '-' * maxwidth) * (self.n - 1)
                boardlines += '\n'

        return label + '\n' + firstline + '\n' + boardlines


    # Cell location utilities
    def illegalCell(self,row,col):
        """ returns True if the (row,column) position is legal for this board,
        regardless of whether the cell is occupied.
        """
        return (row<0) or (col<0) or (row>=self.n) or (col>=self.n)

    def legalCell(self,row,col):
        return not self.illegalCell(row,col)

    # GETTER AND SETTER - FOR INTERNAL USE
    def _getCell(self,row,col,board):
        """get the mark at the specified cell location.
        The cell location is checked for validity and an Exception is raised if the location is not valid."""
        if self.illegalCell(row,col): # illegal location?
            raise Exception('_getCell: Illegal board cell location ({0},{1})'.format(row,col))
            return []
        else:
            if (board == 'User'):
                return self.my_board[row][col]
            elif(board == 'AI'):
                return self.ai_board[row][col]
            else:
                return self.guess_board[row][col]

    def _setCell(self,row,col,mark,board):
        """Set the value of the cell at the specified location to the specified mark. Valid marks are 'S', 'M' and 'H'.
        The cell location is checked for validity, and an exception is raised if it's not valid. This method is low-level and does not check the current value of the cell."""

        if self.illegalCell(row,col): # illegal location ?
            raise Exception('_setCell: Illegal board cell location ({0},{1})'.format(row,col))
        else:
            if mark not in 'SMHX': # illegal mark?
                raise Exception('setCell: invalid mark value {0}'.format(mark))
            else:
                if (board == 'User'):
                    self.my_board[row][col] = mark
                elif(board == 'AI'):
                    self.ai_board[row][col] = mark
                else:
                    self.guess_board[row][col] = mark
        return


    # PUBLIC GAMEPLAY INTERFACES
    def occupied(self,row,col,board):
        """Returns True if the specified cell is occupied and False if not."""
        if self.illegalCell(row,col):
            raise Exception('occupied: illegal cell location ({0},{1})'.format(row,col))
            return False
        else:
            return self._getCell(row,col,board) != ' '

    def unoccupied(self,row,col,board):
        return not self.occupied(row,col,board)

    def placeShip(self, row, col, num, ori, board):
        """Place a ship ('S') at the specified cell location. Raises exceptions if the location is invalid or already occupied."""
        if(ori == 'V'):
            for i in range(num):
                if self.illegalCell(row+i, col):
                    raise Exception('placeShip: Illegal row {0} or col {1}'.format(row+i, col))
                if self._getCell(row+i, col, board) != ' ':
                    raise Exception('placeShip: Cell [{0}][{1}] already occupied by {2}'.format(row+i, col, self._getCell(row+i, col,board)))
            for j in range(num):
                self._setCell(row+j, col, 'X', board)
        elif(ori == 'H'):
            for i in range(num):
                if self.illegalCell(row, col+i):
                    raise Exception('placeShip: Illegal row {0} or col {1}'.format(row, col+i))
                if self._getCell(row, col+i, board) != ' ':
                    raise Exception('placeShip: Cell [{0}][{1}] already occupied by {2}'.format(row, col+i, self._getCell(row, col+i, board)))
            for j in range(num):
                self._setCell(row, col+j, 'X', board)
        else:
            raise Exception(f'Invalid orientation: {ori}')

    def makeGuess(self, row, col, board, output, coordinates):
        """Make a guess at the specified cell location. Marks 'H' for a hit or 'M' for a miss.
        Raises exceptions if the location is invalid or if the cell has already been guessed."""
        if self.illegalCell(row, col):
            raise Exception('makeGuess: Illegal row {0} or col {1}'.format(row, col))
        if self._getCell(row, col, output) in 'MHS':
            raise Exception('makeGuess: Cell [{0}][{1}] already guessed'.format(row, col))
        if self._getCell(row, col, board) == 'X':
            self._setCell(row, col, 'H', output)
            print()
            print(f'Guess: ({row}, {col}) ')
            print('Hit!')
            self._sunkenShipCheck(output, coordinates, row, col)
        else:
            self._setCell(row, col, 'M', output)
            print()
            print(f'Guess: ({row}, {col}) ')
            print('Miss!')

    def _sunkenShipCheck(self, board, coordinates, row, col):
        """Updates guess board for sunken ships"""
        check = True
        for coordinate in coordinates:
            if((row,col)in coordinate):
                true_coordinate = coordinate
                for r,c in coordinate:
                    if self._getCell(r,c,board) == 'H':
                        continue
                    else:
                        check = False
        if check:
            print('Sunken Ship!')
            for r,c in true_coordinate:
                self._setCell(r,c,'S',board)


    def _allShipsSunk(self, board):
        """Internal: If all ships are sunk, return True. Else, return False."""
        if board == 'Guess':
            return sum(row.count('S') for row in self.guess_board) == 17
        else:
            return sum(row.count('S') for row in self.my_board) == 17

    # external high-level state
    def gamestate(self):
        """returns 'You Won!', 'You Lost!', or 'in-progress' based on the state of the board"""
        if self._allShipsSunk('Guess'): return 'You Won!'
        elif self._allShipsSunk('User'): return 'You Lost!'
        else: return 'in-progress'


In [ ]:
# User Set Up
board = Battleship()
ships = [5,4,3,3,2]
user_placements = []
print('Lets begin the game\nThere are 5 ships total of lengths 5, 4, 3, 3, and 2. You will be prompted to enter the row, column, and orientation for each ship\nHere is the empty starting board:')
print(board.__str__('User'))
for ship in ships:
    print(f"Placing a ship of size {ship}.")
    while True:
        row = int(input("Enter the row (0-9) for the ship: "))
        col = int(input("Enter the column (0-9) for the ship: "))
        ori = input("Enter the orientation (H for Horizontal, V for Vertical): ")
        try:
            board.placeShip(row, col,ship,ori,'User')
            placement = []
            for i in range(ship):
                if ori == 'V':
                    placement.append((row+i,col))
                else:
                    placement.append((row,col+i))
            user_placements.append(placement)
            print(board.__str__('User'))
            break
        except Exception as e:
            print(f"Error: {e}")

In [ ]:
# Randomization of AI's Board
import random
print("Building the AI's board")
ai_placements = []
ori_options = ['V','H']

for ship in ships:
    while True:
        row = random.randint(0,9)
        col = random.randint(0,9)
        ori = random.randint(0,1)
        try:
            board.placeShip(row, col,ship,ori_options[ori],'AI')
            placement = []
            for i in range(ship):
                if ori_options[ori] == 'V':
                    placement.append((row+i,col))
                else:
                    placement.append((row,col+i))
            ai_placements.append(placement)
            break
        except:
            continue


In [ ]:
from ast import JoinedStr
from importlib import import_module
def ai_hit_search(board):
    for num_row,row in enumerate(board.my_board):
        for num_col,symbol in enumerate(row):
            if symbol == 'H':
                hit_row = num_row
                hit_col = num_col
                break
        if symbol == 'H':
            break
    if symbol == 'H':
        # horizontal guess
        if num_col < 9:
            if board._getCell(num_row,num_col+1,'User') == 'H' and num_col>0:
                if board._getCell(num_row,num_col-1,'User') != 'M' and board._getCell(num_row,num_col-1,'User') != 'S':
                    return (num_row,num_col-1)
            if board._getCell(num_row,num_col+1,'User') == 'H':
                i=0
                while board._getCell(num_row,num_col+i,'User') == 'H':
                    i+=1
                    if(num_col+i>9):
                        break
                if(num_col+i<=9):
                    if board._getCell(num_row,num_col+i,'User') != 'M' and board._getCell(num_row,num_col+i,'User') != 'S':
                        return (num_row,num_col+i)
        # vertical
        j = 0
        while board._getCell(num_row+j,num_col,'User') == 'H':
            j-=1
            if num_row+j < 0:
                break
        if num_row+j >= 0:
            if board._getCell(num_row+j,num_col,'User') != 'M' and board._getCell(num_row+j,num_col,'User') != 'S':
                return (num_row+j,num_col)
        j+=1
        while board._getCell(num_row+j,num_col,'User') == 'H':
            j+=1
            if num_row+j > 9:
                break
        if num_row+j<=9:
            if board._getCell(num_row+j,num_col,'User') != 'M' and board._getCell(num_row+j,num_col,'User') != 'S':
                return (num_row+j,num_col)
        # pick neighbors
        if num_col < 9:
            if board._getCell(num_row,num_col+1,'User') != 'M' and board._getCell(num_row,num_col+1,'User') != 'S':
                return (num_row,num_col+1)
        if num_row > 0:
            if board._getCell(num_row-1,num_col,'User') != 'M' and board._getCell(num_row-1,num_col,'User') != 'S':
                return (num_row-1,num_col)
        if num_col > 0:
            if board._getCell(num_row,num_col-1,'User') != 'M' and board._getCell(num_row,num_col-1,'User') != 'S':
                return (num_row,num_col-1)
        if num_row < 9:
            if board._getCell(num_row+1,num_col,'User') != 'M' and board._getCell(num_row+1,num_col,'User') != 'S':
                return (num_row+1,num_col)
        return ()

    else:
        return ()


In [ ]:
# gameplay
turn = 1
print("Lets start the gameplay!")
print(board.__str__('Guess'))
numbers = [0,1,2,3,4,5,6,7,8,9]
num_weights = [1,1,1,1.5,1.5,1.5,1.5,1,1,1]
while board.gamestate() == 'in-progress':
    if turn == 1:
        while True:
            row = int(input('Which row would you like to guess?: '))
            col = int(input('Which column would you like to guess?: '))
            try:
                board.makeGuess(row,col,'AI','Guess',ai_placements)
                print(board.__str__('Guess'))
                break
            except:
                print('Invalid Input. Try Again!')
                continue
        turn = 2
    else:
        while True:
            row = random.choices(numbers, weights = num_weights)[0]
            col = random.choices(numbers, weights = num_weights)[0]
            search = ai_hit_search(board)
            if search != ():
                row,col = ai_hit_search(board)
                try:
                    board.makeGuess(row,col,'User','User',user_placements)
                    print(board.__str__('User'))
                    break
                except:
                    continue

            else:
                try:
                    board.makeGuess(row,col,'User','User',user_placements)
                    print(board.__str__('User'))
                    break
                except:
                    continue
        turn = 1

print(board.gamestate())